In [1]:
import re

import pandas as pd
from bs4 import BeautifulSoup
import requests
import pymysql

In [2]:
search = input('검색할 키워드를 입력해주세요.')

page = int(input("크롤링할 페이지를 입력해주세요. ex)1(숫자만입력):"))  # ex)1 =1페이지,2=2페이지...
print("크롤링할 페이지: ", page, "페이지")

크롤링할 페이지:  1 페이지


In [3]:
#start수를 1, 11, 21, 31 ...만들어 주는 함수
page_num = 0

if page == 1:
    page_num = 1
elif page == 0:
    page_num = 1
else:
    page_num = page + 9 * (page - 1)

#url 생성
url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page_num)
print("생성url: ", url)

생성url:  https://search.naver.com/search.naver?where=news&sm=tab_pge&query=코로나&start=1


In [4]:
#html불러오기
original_html = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
html = BeautifulSoup(original_html.text, "html.parser")

In [5]:
# 검색결과
articles = html.select("div.group_news > ul.list_news > li div.news_area > a")
print(articles)
# 검색된 기사의 갯수
print(len(articles), "개의 기사가 검색됨.")

[<a class="news_tit" href="http://www.newsis.com/view/?id=NISX20220330_0001813576&amp;cID=10201&amp;pID=10200" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=1&amp;i=88000127_000000000000000011092452&amp;g=003.0011092452&amp;u='+urlencode(this.href));" target="_blank" title='정부 "요양병원에 코로나 치료제 최우선 처방"'>정부 "요양병원에 <mark>코로나</mark> 치료제 최우선 처방"</a>, <a class="news_tit" href="https://news.jtbc.joins.com/article/article.aspx?news_id=NB12053083" onclick="return goOtherCR(this, 'a=nws*j.tit&amp;r=3&amp;i=08138260_000000000000000000294034&amp;g=437.0000294034&amp;u='+urlencode(this.href));" target="_blank" title="오늘부터 코로나 확진자도 동네 병·의원서 대면진료 가능">오늘부터 <mark>코로나</mark> 확진자도 동네 병·의원서 대면진료 가능</a>, <a class="news_tit" href="http://yna.kr/AKR20220330112351001?did=1195m" onclick="return goOtherCR(this, 'a=nws*h.tit&amp;r=4&amp;i=880000D8_000000000000000013082356&amp;g=001.0013082356&amp;u='+urlencode(this.href));" target="_blank" title='인수위 "코로나 확산 정점 지나면 영업시간 제한 즉시 철폐"(종합)'>인수위 "<mark>코로나</mark> 확산

In [6]:
#뉴스기사 제목 가져오기
# Q1. 여기서 제목과 특정 행동과의 상관관계를 구하자

news_title = []
for i in articles:
    news_title.append(i.attrs['title'])
news_title

['정부 "요양병원에 코로나 치료제 최우선 처방"',
 '오늘부터 코로나 확진자도 동네 병·의원서 대면진료 가능',
 '인수위 "코로나 확산 정점 지나면 영업시간 제한 즉시 철폐"(종합)',
 '79세 고령 바이든, 4차 코로나 백신 공개 접종한다',
 "[단독] 코로나로 주민들 신음하는데...의정부시, '외유성 해외연수' 추진 논란",
 '인수위, 정부에 “내주까지 소상공인·자영업자 코로나 손실 규모 보고하라”',
 '질병청 "코로나19 항체조사 전국서 시행‥소아·청소년도 포함"',
 '인수위 “코로나 감소세 들어서면 거리두기 완화해야”',
 '카페 1회용품 제한… 코로나 상황 개선까지 ‘계도’만 한다',
 '"코로나19, 종식 안 되고 소아과 질환 토착화할 수도"']

In [7]:
#뉴스기사 URL 가져오기
# 상위 10개 뉴스기사
# 어떤 회사에서 뉴스를 많이 낼 때 어떠한 행동이 나타나는지 인사이트를 구해보자.

news_url = []
for i in articles:
    news_url.append(i.attrs['href'])
news_url

['http://www.newsis.com/view/?id=NISX20220330_0001813576&cID=10201&pID=10200',
 'https://news.jtbc.joins.com/article/article.aspx?news_id=NB12053083',
 'http://yna.kr/AKR20220330112351001?did=1195m',
 'http://www.edaily.co.kr/news/newspath.asp?newsid=01239846632268896',
 'https://www.ytn.co.kr/_ln/0103_202203310455519362',
 'https://biz.chosun.com/policy/politics/2022/03/30/QBJYD4QRORDHTFNRFYIMCFNYJQ/?utm_source=naver&utm_medium=original&utm_campaign=biz',
 'https://imnews.imbc.com/news/2022/society/article/6354801_35673.html',
 'https://www.chosun.com/national/welfare-medical/2022/03/30/HG5RFFA5BJDI3EGGBGQWDCYMXM/?utm_source=naver&utm_medium=referral&utm_campaign=naver-news',
 'http://www.segye.com/content/html/2022/03/30/20220330514917.html?OutUrl=naver',
 'http://www.newsis.com/view/?id=NISX20220330_0001813814&cID=13101&pID=13100']

기사 내용 적재

In [8]:
contents = []
for url, title in zip(news_url, news_title):
    #각 기사 html get하기
    news = requests.get(url)  # url 리스트에서 하나 가져옴
    news.encoding = 'utf-8'  # 크롤링 시 텍스트 인코딩
    news_html = BeautifulSoup(news.text, "html.parser")  # 어떻게 가져올지

    text = news_html.find_all('p')  # p 태그 전부 찾기

    craw_list = text[:]  # 가져온 p 태그 리스트 list type으로 저장

    craw_list_str = []
    for element in craw_list:
        craw_list_str.append(element.text)  # p 태그의 text 부분만 정제

    contents.append([title, craw_list_str])  # 제목과 기사 내용 mapping 해서 contents 변수에 저장
contents

[['정부 "요양병원에 코로나 치료제 최우선 처방"',
  ['2022.03.31 (목)',
   '서울 9℃',
   '사회',
   '정부 "요양병원에 코로나 치료제 최우선 처방"',
   '등록 2022.03.30 12:11:31수정 2022.03.30 17:22:41',
   '기사내용 요약',
   '\r\n팍스로비드 최우선 처방…라게브리오도 활용\r\n요양시설 고령층의 경우 경증이라도 병원 이송\r\n위중증 막기 위해 중증환자는 핫라인으로 전원 \t\t\t\t\t\t\t\r\n\t\t\t\t\t\t',
   '[서울=뉴시스] 조수정 기자 = 지난 25일 오전 서울 중구 서울역 임시선별검사소에서 한 시민이 PCR 검사를 받고 있다. 이날 사망은 393명, 위중증 환자는 1085명으로 집계됐다. 2022.03.25. chocrystal@newsis.com',
   '[서울=뉴시스] 조수정 기자 = 지난 25일 오전 서울 중구 서울역 임시선별검사소에서 한 시민이 PCR 검사를 받고 있다. 이날 사망은 393명, 위중증 환자는 1085명으로 집계됐다. 2022.03.25. chocrystal@newsis.com',
   'Copyright © NEWSIS.COM, 무단 전재 및 재배포 금지',
   '뉴시스 기획특집',
   '일회용 금지 강행',
   '윤석열 시대 부동산',
   '경유값 폭등',
   '피플인사이드',
   '"\'돼지의왕\' 김동욱\xa0\xa0사려깊은 연기 감탄"',
   '연상호 감독',
   '"9살~72살까지 연기\xa0나를 돌아보는 작품"',
   '배우 윤나무',
   '인기 테마포토',
   '\n\r\n\t\t\t\t\t11년만에 이란 꺾은 벤투호\t\t\t\t\n',
   '\n\r\n\t\t\t\t\t박근혜 전 대통령 퇴원\t\t\t\t\n',
   '\n\r\n\t\t\t\t\t尹, 정순택 대주교 만나 차담회\t\t\t\t\n',
   "\n\r\n\t\t\t\t\t봄의 전령 진달래 '활짝'\t\t\t\t\n",


In [17]:
news = requests.get('http://www.newsis.com/view/?id=NISX20220330_0001813814&cID=13101&pID=13100')
news.encoding = 'utf-8'
news_html = BeautifulSoup(news.text, "html.parser")
#기사 내용 가져오기 (p태그의 내용 모두 가져오기)
# 기사를 가져오고 보기 편하도록 정재하여 사용하여야한다.
text = news_html.find_all('p')

In [18]:
craw_list = text[:]
type(craw_list)

list

In [19]:
craw_list_str = []

for element in craw_list:
    craw_list_str.append(element.text)

craw_list_str

['2022.03.31 (목)',
 '서울 13℃',
 'IT·바이오',
 '"코로나19, 종식 안 되고 소아과 질환 토착화할 수도"',
 '등록 2022.03.30 15:02:57수정 2022.03.30 20:13:44',
 '기사내용 요약',
 '\r\n"40년간 코로나 바이러스 소아 중심으로 토착화 전례"\r\n"전파력 높고 중증도 낮은 변이가 우세종으로 남을 것"\r\n"5~11세 백신 접종 필요성은 종합적인 분석 필요"\r\n"\n',
 '[서울=뉴시스] 권창회 기자 =\xa0 만 5~11세 소아 대상 코로나19 백신 예방접종 사전예약일인 24일 오후 서울 시내 한 소아청소년과의원에서 간호사가 유소아 대상 코로나19 백신 접종 예약 안내문을 부착하고 있다. 접종 시행 위탁의료기관은 총 1200곳이 선정됐으며, 코로나19 예방접종 홈페이지에서 확인할 수 있다. 2022.03.24. kch0523@newsis.com',
 '[서울=뉴시스] 권창회 기자 =\xa0 만 5~11세 소아 대상 코로나19 백신 예방접종 사전예약일인 24일 오후 서울 시내 한 소아청소년과의원에서 간호사가 유소아 대상 코로나19 백신 접종 예약 안내문을 부착하고 있다. 접종 시행 위탁의료기관은 총 1200곳이 선정됐으며, 코로나19 예방접종 홈페이지에서 확인할 수 있다. 2022.03.24. kch0523@newsis.com',
 'Copyright © NEWSIS.COM, 무단 전재 및 재배포 금지',
 '관련기사',
 '뉴시스 기획특집',
 '일회용 금지 강행',
 '윤석열 시대 부동산',
 '경유값 폭등',
 '피플인사이드',
 '"\'돼지의왕\' 김동욱\xa0\xa0사려깊은 연기 감탄"',
 '연상호 감독',
 '"9살~72살까지 연기\xa0나를 돌아보는 작품"',
 '배우 윤나무',
 '인기 테마포토',
 '\n\r\n\t\t\t\t\t11년만에 이란 꺾은 벤투호\t\t\t\t\n',
 '\n\r\n\t\t\t\t\t박근혜 전 대통령 퇴원\t\t\t\t\n',
 '\n\r\

In [12]:
# db에 id, date, title, keyword, topic, company,contents 로 설계 후 적재
# 적재를 하기 전에 자연어 처리를 통한 좋음 안 좋음을 수치로 표기해야함
# keyword은 검색어를 넣기
# 우선 토픽은 크게 중요하지 않으니 잠깐 배제 후 topic 분류 알고리즘 탐색
# 기본적으로는 non topic news data 테이블 사용
# 넣기 전에 해야할 목록

# 1. 기능 함수화
# 2. db에 들어갈 값들을 각 컬럼에 맞게 전처리
# 3. contents 요약